## GridSearch For Best UMAP and HDBSCAN hyperparameters for PubMed abstracts
This notebook documents partially GridSearch for the optimal UMAP and HDBSCAN parameters for our corpus of PubMed abstracts in our topic modelling exploration. 

The hyperparameters tested were: n_components and n_neighbors for UMAP, and min_cluster_size and min_samples for HDBSCAN.  
*n_components* is essentially the dimension size after dimensionality reduction. Although the default is 5, I swept values between 5-50. Somewhere around 20 is the sweet spot.
*n_neighbors* control the balance between local and global structure. Small values fragment larger topics, and large values produce more cohesive, well-separated clusters. The UMAP documentation suggests increasing up to 30 when using UMAP as a preprocessing step for density-based clustering (https://umap-learn.readthedocs.io/en/latest/clustering.html)  
*min_cluster_size* and *min_samples* - based on the BERTopic documentation (https://maartengr.github.io/BERTopic/getting_started/parameter%20tuning/parametertuning.html#metric) min_samples should be set significantly below min_cluster_size to reduce noise.

To my knowledge, there are no previous studies on hyperparameter tuning for BERTopic for topic modelling on PubMed abstracts, so we have little to go by for starting points, which is why I deployed a large range for the hyperparameter search.

The metric chosen was Density-Based Clustering Validation (DBCV), which is an intrinsic evaluation metric for clustering. DBCV compares intra-cluster density against inter-cluster density. Scores range from 1 (dense, well-separated) to -1 (points are closer to neighboring clusters than their own). A value of zero is essentially random, without any structure. DBCV is particularly suitable for UMAP and HDBSCAN as it evaluates cluster quality directly in the reduced embedding space without requiring ground-truth labels.  

The search started with a wide range for UMAP, in steps of 5, then progressively narrowing down both range and steps. For each iteration, a range of 2 around the best hyperparameter value was tested. For example, if DBCV was highest at n_components = 21 when tested for 10,15,20,25,30 then it was retested for n_components 19,20,21,22,23.  

UMAP was first tested, and a range around the optimal n_components and n_neighbor combination used to test against HDBSCAN hyperparameters.
The cells below show the last iterations performed.

In [1]:
# Check that GPU is available for use
import torch
torch.cuda.is_available()

True

In [2]:
# Instantiate the embedding model
from sentence_transformers import SentenceTransformer
embed_model = SentenceTransformer('neuml/pubmedbert-base-embeddings', device='cuda')

In [3]:
# load data
import pandas as pd
data = pd.read_csv('pubmed_abstracts.csv')
to_drop = ['Title','pmid','meshMajor', 'meshid', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'L', 'M', 'N', 'Z']
data = data.drop(to_drop, axis=1)
data = data.sample(n=5000, random_state=42)
data = data.reset_index(drop=True)
data['abstractText'] = data['abstractText'].str.lower()

# A bit of cleaning to remove numbers
import re 
def remove_numbers(series):
    def rem_no(text):
        pattern = r'\b\d+(\.\d{1,2})?\b'
        cleaned_text = re.sub(pattern, '', text)
        cleaned_text = cleaned_text.strip()
        return cleaned_text
    return series.apply(rem_no)

# Extract out just the abstracts to a list - BERTopic works with lists
data['no_numbers'] = remove_numbers(data['abstractText'])
abstracts = data['no_numbers'].to_list()

In [4]:
# generate the embeddings
embeddings = embed_model.encode(abstracts)

In [14]:
# Do the GridSearch for UMAP first

from bertopic import BERTopic
from umap import UMAP
from tqdm import tqdm
from hdbscan import HDBSCAN

dimensions = [20, 21, 22, 23, 24, 25]
results = []

for dimension in tqdm(dimensions, desc='UMAP n_components sweep'):
    print(f'\n n_components = {dimension}')

    # UMAP
    reducer = UMAP(
        n_components = dimension,
        n_neighbors = 15,
        min_dist = 0.0,
        metric = 'cosine',
        random_state = 42,
        verbose = False
    )
    reduced = reducer.fit_transform(embeddings)

    # HDBSCAN
    clusterer = HDBSCAN(
        min_cluster_size=10,
        min_samples = 5,
        metric = 'euclidean',
        cluster_selection_method = 'eom',
        gen_min_span_tree = True # required for dbcv (relative validity_)
    )
    clusterer.fit(reduced)

    labels = clusterer.labels_
    n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
    n_noise = int((labels == -1).sum())
    noise_pct = 100 * n_noise / len(labels)
    dbcv = clusterer.relative_validity_

    print(f' Clusters : {n_clusters}')
    print(f' Noise    : {n_noise}{noise_pct:.1f}%')
    print(f' DBCS     : {dbcv:.4f}')

    results.append({
        'n_components' : dimension,
        'n_clusters' : n_clusters,
        'n_noise' : n_noise,
        'noise_pct' : round(noise_pct, 2),
        'dbcv' : round(dbcv, 4)
    })

print('\n  SUMMARY: ---------------------------')
df = pd.DataFrame(results).set_index('n_components')
best = df['dbcv'].idxmax()
print(f"\nBest n_components by DBCV: {best}  (DBCV = {df.loc[best, 'dbcv']})")


UMAP n_components sweep:   0%|          | 0/4 [00:00<?, ?it/s]


 n_components = 22


UMAP n_components sweep:  25%|██▌       | 1/4 [00:05<00:17,  5.70s/it]

 Clusters : 130
 Noise    : 169133.8%
 DBCS     : 0.2099

 n_components = 23


UMAP n_components sweep:  50%|█████     | 2/4 [00:11<00:11,  5.84s/it]

 Clusters : 121
 Noise    : 154230.8%
 DBCS     : 0.1857

 n_components = 24


UMAP n_components sweep:  75%|███████▌  | 3/4 [00:16<00:05,  5.52s/it]

 Clusters : 126
 Noise    : 161932.4%
 DBCS     : 0.1192

 n_components = 25


UMAP n_components sweep: 100%|██████████| 4/4 [00:22<00:00,  5.56s/it]

 Clusters : 129
 Noise    : 173934.8%
 DBCS     : 0.1474

  SUMMARY: ---------------------------

Best n_components by DBCV: 22  (DBCV = 0.2099)


In [21]:
from itertools import product
dimensions = range(19,25)
neighbors = range(5,10)
results = []

for n_components, n_neighbors in tqdm(list(product(dimensions, neighbors))):
    reducer = UMAP(
        n_components=n_components,
        n_neighbors=n_neighbors,
        min_dist=0.0,
        metric="cosine",
        random_state=42,
    )
    reduced = reducer.fit_transform(embeddings)

    clusterer = HDBSCAN(
        min_cluster_size=10,
        min_samples=5,
        gen_min_span_tree=True,
    ).fit(reduced)

    labels = clusterer.labels_
    n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
    noise_pct  = 100 * (labels == -1).sum() / len(labels)

    results.append({
        "n_components": n_components,
        "n_neighbors":  n_neighbors,
        "n_clusters":   n_clusters,
        "noise_pct":    round(noise_pct, 2),
        "dbcv":         round(clusterer.relative_validity_, 4),
    })

df = pd.DataFrame(results)
print(df.sort_values("dbcv", ascending=False).to_string(index=False))

100%|██████████| 30/30 [03:36<00:00,  7.20s/it]

 n_components  n_neighbors  n_clusters  noise_pct   dbcv
           24            5         177      18.20 0.2358
           22            6         156      21.74 0.2274
           21            6         159      20.46 0.2184
           22            5         172      20.22 0.2159
           20            6         157      22.16 0.2122
           22            7         157      22.84 0.2115
           23            6         169      23.26 0.2089
           23            7         148      23.70 0.2063
           19            7         153      23.96 0.2035
           20            7         152      24.66 0.1977
           19            8         149      26.32 0.1927
           24            6         161      22.56 0.1901
           20            8         146      27.38 0.1887
           24            7         150      22.66 0.1884
           20            5         179      20.86 0.1836
           20            9         131      30.32 0.1833
           19            5     

The best combination of dimensions and neighbors is 20, 10.

In [5]:
# apply the best combination for dimensionality reduction

reducer = UMAP(
        n_components=20,
        n_neighbors=10,
        min_dist=0.0,
        metric="cosine",
        random_state=42,
    )

reduced = reducer.fit_transform(embeddings)

In [25]:
# Do the GridSearch for HDBSCAN

MIN_CLUSTER_SIZES   = range(10,51)
MIN_SAMPLES_LIST    = range(3,16)

results = []

for min_cluster_size, min_samples in tqdm(
    list(product(MIN_CLUSTER_SIZES, MIN_SAMPLES_LIST))
):
    # Skip nonsensical combinations
    if min_samples > min_cluster_size:
        continue

    clusterer = HDBSCAN(
        min_cluster_size=min_cluster_size,
        min_samples=min_samples,
        cluster_selection_method='eom',
        metric="euclidean",
        gen_min_span_tree=True,
    ).fit(reduced)

    labels = clusterer.labels_
    n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
    noise_pct  = 100 * (labels == -1).sum() / len(labels)

    results.append({
        "min_cluster_size": min_cluster_size,
        "min_samples":      min_samples,
        "n_clusters":       n_clusters,
        "noise_pct":        round(noise_pct, 2),
        "dbcv":             round(clusterer.relative_validity_, 4),
    })

df = pd.DataFrame(results)
print(df.sort_values("dbcv", ascending=False).head(15).to_string(index=False))

100%|██████████| 533/533 [03:46<00:00,  2.36it/s]

 min_cluster_size  min_samples  n_clusters  noise_pct   dbcv
               25            5          61      28.26 0.3080
               20           13          62      37.36 0.3072
               11            5         117      28.44 0.2995
               23            5          66      27.44 0.2993
               24            5          63      28.34 0.2926
               13            5         100      29.42 0.2879
               35            7          36      26.40 0.2838
               19           13          66      37.40 0.2814
               26            5          56      28.68 0.2803
               14            5          96      29.78 0.2797
               37            7          32      25.42 0.2783
               36            7          32      25.42 0.2783
               50           12          27      29.34 0.2775
               19            5          77      28.78 0.2775
               49           12          27      29.34 0.2775


In [6]:
clusterer = HDBSCAN(
        min_cluster_size=25,
        min_samples=5,
        cluster_selection_method='eom',
        metric="euclidean",
        gen_min_span_tree=True,
    ).fit(reduced)

The best parameters are:  
UMAP: n_components = 20, n_neighbors = 10  
HDBSCAN: min_cluster_size = 25, min_sample = 10  
We'll fit these into our BERTopic pipeline.